In [1]:
import nltk
from nltk.corpus import treebank 
from nltk.tree import Tree
import csv 
import pandas as pd
import numpy as np
import seaborn as sns
import string
import re
import math
import sklearn
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls "/content/drive/My Drive/Colab Notebooks/data"

adversarial_swap_train.csv  adversarial_swap_train_final.csv


In [15]:
df = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/data/adversarial_swap_train_final.csv")
df = df.dropna()
df.head(10)

,Unnamed: 0,text,sentiment,classification,clean_text
0,1,Sooo SAD I will miss you here in San Diego!!!,negative,0.0,sooo sad miss san diego
1,2,my boss is bullying me...,negative,0.0,bos bullying
2,3,what interview! leave me alone,negative,0.0,interview leave alone
3,4,"Sons of ****, why couldn`t they put them on t...",negative,0.0,son couldnt put release already bought
4,6,2am feedings for the baby are fun when he is a...,positive,1.0,feeding baby fun smile coo
5,9,Journey!? Wow... u just became cooler. hehe....,positive,1.0,journey wow u became cooler hehe possible
6,11,I really really like the song Love Story by Ta...,positive,1.0,really really like song love story taylor swift
7,12,My Sharpie is running DANGERously low on ink,negative,0.0,sharpie running dangerously low ink
8,13,i want to go to music tonight but i lost my vo...,negative,0.0,want go music tonight lost voice
9,15,"Uh oh, I am sunburned",negative,0.0,uh oh sunburned


In [4]:
!pip3 install transformers
import transformers
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)

In [9]:
def encode_for_bert(data, maximum_length):
    input_ids = []
    attention_masks = []
  
    for sentence in data:
        if isinstance(sentence, float):
          print(sentence)
        encoded = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=maximum_length, pad_to_max_length=True, return_attention_mask=True, truncation=True)
      
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return np.array(input_ids),np.array(attention_masks)

In [16]:
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(df['clean_text'], df['classification'], test_size=0.2)
train_input_ids, train_attention_masks = encode_for_bert(X_train_raw, 23)
test_input_ids, test_attention_masks = encode_for_bert(X_test_raw, 23)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [17]:
def add_sparse_layers(bert_model):
    input_ids = tf.keras.Input(shape=(23,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(23,),dtype='int32')
  
    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer='l1')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)

    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=6e-6), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [18]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-large-uncased')

Some layers from the model checkpoint at bert-large-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-large-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [19]:
model = add_sparse_layers(bert_model)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  335141888   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 23,                                            

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
history = model.fit([train_input_ids,train_attention_masks], y_train_raw, validation_split=0.2, epochs=2,batch_size=10)

Epoch 1/2
1047/1047 [==============================] - 633s 567ms/step - loss: 12.0797 - accuracy: 0.8213 - val_loss: 11.2645 - val_accuracy: 0.8950
Epoch 2/2
1047/1047 [==============================] - 586s 559ms/step - loss: 10.5162 - accuracy: 0.9113 - val_loss: 9.8043 - val_accuracy: 0.9041


In [21]:
result = model.predict([test_input_ids,test_attention_masks])
result = np.round(result).astype(int)

In [22]:
from sklearn.metrics import accuracy_score, f1_score

In [23]:
print(accuracy_score(y_test_raw, result))
print(f1_score(y_test_raw, result))

0.8994498777506112
0.9068780073591848


In [32]:
def get_base_model(bert_model):
    input_ids = tf.keras.Input(shape=(23,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(23,),dtype='int32')
  
    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    output = tf.keras.layers.Dropout(0.4)(output)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=6e-6), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [33]:
base_model = get_base_model(bert_model)
base_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  335141888   ['input_5[0][0]',                
                                thPoolingAndCrossAt               'input_6[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 23,                                          

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [34]:
base_history = base_model.fit([train_input_ids,train_attention_masks], y_train_raw, validation_split=0.2, epochs=2,batch_size=10)

Epoch 1/2
1047/1047 [==============================] - 627s 560ms/step - loss: 3.2913 - accuracy: 0.7769 - val_loss: 0.4534 - val_accuracy: 0.8900
Epoch 2/2
1047/1047 [==============================] - 580s 554ms/step - loss: 3.3375 - accuracy: 0.7683 - val_loss: 0.3651 - val_accuracy: 0.8858


In [35]:
base_result = base_model.predict([test_input_ids,test_attention_masks])
base_result = np.round(result).astype(int)

In [ ]:
print(accuracy_score(y_test_raw, base_result))
print(f1_score(y_test_raw, base_result))